<img src="./airbnb.png">
<br />
<div style="text-align: center;">
    <span style="font-weight: bold; color:#FF7674; font-family: 'Courier New'; font-size: 5em;">Data Modeling<br><br> AirBnB Madrid</span>
</div>

<br />
<div style="text-align: left;">
    <span style="color:#8585BB; font-family: 'Courier New'; font-size: 1.5em;">Para esta parte de nuestro trabajo utilizaremos la librería sklearn<br><br>Se aplicarán los conceptos de Regresiones para crear modelos que nos permita determinar los factores que afectan el precio de alquiler de los pisos en AirBnB en la ciudad de Madrid.
    <br><br>Procedamos cargando las librerías necesarias y avancemos:
    <br></span>
</div>

In [1]:
import sys
import datetime as dt                  

#import seaborn as sns
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import ipywidgets as widgets

from matplotlib.dates import YearLocator, MonthLocator, DateFormatter, WeekdayLocator, DayLocator
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import Imputer, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Python default packages
from collections import defaultdict
import re
import warnings
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print('Python version: ', sys.version)
print('Pandas version: ', pd.__version__)
print('Today: ', dt.date.today())

('Python version: ', '2.7.12 |Anaconda custom (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]')
('Pandas version: ', u'0.18.1')
('Today: ', datetime.date(2017, 7, 14))


In [2]:
# Usamos el fichero que ya hemos limpiado
listing = pd.read_csv('./listing_clean.csv')

In [3]:
listing.head(3)

,Unnamed: 0,id,name,host_id,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,price_log,distancia_metro
0,0,844856,BEST CENTER LOCATED IN MADRID,4410687,Luis Manuel,2012-12-15,False,1.0,1.0,"['email', 'phone', 'reviews']",True,False,Centro,28013,40.417070,-3.708815,Apartment,Entire home/apt,5,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air conditio...",62.0,400.0,1600.0,NaN,30.0,3,10.0,1125,1125,yesterday,0,0,0,0,2,2016-08-10,2016-08-19,100.0,8.0,6.0,10.0,10.0,10.0,8.0,True,False,strict,False,False,1,0.25,4.127134,0.0
1,1,4693891,PASEO de la CASTELLANA EXCELLENT,8879142,Ana,2013-09-16,False,6.0,6.0,"['email', 'phone', 'reviews', 'manual_offline']",True,True,Tetuan,28029,40.468373,-3.689958,Apartment,Private room,2,2.0,1.0,2.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Wheelchair a...",32.0,168.0,598.0,NaN,NaN,1,15.0,1,90,3 months ago,0,2,32,307,39,2014-12-03,2016-12-27,85.0,9.0,9.0,9.0,9.0,9.0,9.0,True,False,moderate,False,False,6,1.36,3.465736,0.0
2,2,14063097,Habitación individual/cama doble/ventana interior,84316058,Mara,2016-07-17,False,5.0,5.0,"['email', 'phone', 'reviews']",True,False,Centro,28004,40.423427,-3.703112,Apartment,Private room,1,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",42.0,NaN,NaN,NaN,NaN,1,0.0,1,1125,2 weeks ago,0,1,18,19,1,2016-10-25,2016-10-25,100.0,10.0,10.0,10.0,10.0,10.0,10.0,True,False,strict,False,False,5,0.18,3.737670,0.0


In [4]:
listing['distancia_metro'] = listing['distancia_metro'].apply(lambda x: x*1000)
listing['distancia_metro'] = listing['distancia_metro'].round(2)

In [5]:
listing.head(3)

,Unnamed: 0,id,name,host_id,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,price_log,distancia_metro
0,0,844856,BEST CENTER LOCATED IN MADRID,4410687,Luis Manuel,2012-12-15,False,1.0,1.0,"['email', 'phone', 'reviews']",True,False,Centro,28013,40.417070,-3.708815,Apartment,Entire home/apt,5,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air conditio...",62.0,400.0,1600.0,NaN,30.0,3,10.0,1125,1125,yesterday,0,0,0,0,2,2016-08-10,2016-08-19,100.0,8.0,6.0,10.0,10.0,10.0,8.0,True,False,strict,False,False,1,0.25,4.127134,0.0
1,1,4693891,PASEO de la CASTELLANA EXCELLENT,8879142,Ana,2013-09-16,False,6.0,6.0,"['email', 'phone', 'reviews', 'manual_offline']",True,True,Tetuan,28029,40.468373,-3.689958,Apartment,Private room,2,2.0,1.0,2.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Wheelchair a...",32.0,168.0,598.0,NaN,NaN,1,15.0,1,90,3 months ago,0,2,32,307,39,2014-12-03,2016-12-27,85.0,9.0,9.0,9.0,9.0,9.0,9.0,True,False,moderate,False,False,6,1.36,3.465736,0.0
2,2,14063097,Habitación individual/cama doble/ventana interior,84316058,Mara,2016-07-17,False,5.0,5.0,"['email', 'phone', 'reviews']",True,False,Centro,28004,40.423427,-3.703112,Apartment,Private room,1,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",42.0,NaN,NaN,NaN,NaN,1,0.0,1,1125,2 weeks ago,0,1,18,19,1,2016-10-25,2016-10-25,100.0,10.0,10.0,10.0,10.0,10.0,10.0,True,False,strict,False,False,5,0.18,3.737670,0.0


<br><div style="text-align: left;">
    <span style="font-weight: bold; color:#FF7674; font-family: 'Courier New'; font-size: 2em;">Modelando el precio </span>
</div>


<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">En esta sección, se creará una serie de modelos de regresión para intentar predecir el precio de los pisos basado en las características de nuestros datos. Vamos a probar y comparar una regresión lineal, Ridge Regression y un Random Forest. 
<br><br>
Primero, imputamos los missing values en las columnas donde puedan existir para evitar cualquier problema. Utilizamos una estrategia de imputación de medianas porque es más robusta ante los outliers que si usaramos la media. 
<br><br>
Finalmente, las variables categóricas se convierten en variables numéricas con el comando get_dummies de pandas para que puedan ser aceptadas por nuestra librería de python y poder ejecutar los modelos.
<br><br>
Los datos se dividen en train y test (80/20) para que los modelos puedan ser validados en datos de prueba no vistos. Dado que el precio de los pisos siempre debe ser positivo, la variable de respuesta tomada para los modelos de regresión es ln(price). Esto significa que las predicciones finales serán la exponencial de x donde x es la salida del modelo de predicción.
</span>
</div>

In [6]:
features = ['neighbourhood_group_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 
            'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'instant_bookable', 'distancia_metro']
categorical = ['neighbourhood_group_cleansed', 'room_type']
numerical = ['accommodates', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'distancia_metro']

X = listing[features]
y = np.log(listing['price'])

print 'Imputando Missing Values...'
Imp = Imputer(strategy='median')
X.loc[:,['bathrooms', 'bedrooms', 'review_scores_rating', 'distancia_metro']] = Imp.fit_transform(X.loc[:, ['bathrooms', 'bedrooms', 'review_scores_rating', 'distancia_metro']])

print 'Escalando variables numericas...' 
d = defaultdict(MinMaxScaler)
X.loc[:, numerical] = X.loc[:, numerical].apply(lambda x: d[x.name].fit_transform(x))

print 'Creando variable dummy...' 
X = pd.get_dummies(X, columns=categorical)

print 'Separando data en Train y Test...'
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.2, random_state=1994)

print 'Fit Linear Regression...' 
lm = LinearRegression()
lm.fit(train_X, train_Y)

print 'Fit Ridge Regression...'
ridge = Ridge()
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]}
gs = GridSearchCV(ridge, param_grid)
gs.fit(train_X, train_Y)
ridge = gs.best_estimator_
ridge.fit(train_X, train_Y)

print 'Fit Random Forest...' 
rf = RandomForestRegressor(n_estimators=1000, max_depth=16, min_samples_leaf=3)

rf.fit(train_X, train_Y)

Imputando Missing Values...
Escalando variables numericas...
Creando variable dummy...
Separando data en Train y Test...
Fit Linear Regression...
Fit Ridge Regression...
Fit Random Forest...


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=3,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [7]:
# Obtain predictions and scores on the test set
ridge_pred = ridge.predict(test_X)
ridge_score = r2_score(np.exp(test_Y), np.exp(ridge_pred))

lm_pred = lm.predict(test_X)
lm_score = r2_score(np.exp(test_Y), np.exp(lm_pred))

rf_pred = rf.predict(test_X)
rf_score = r2_score(np.exp(test_Y), np.exp(rf_pred))

In [8]:
print 'Regresión lineal', round(lm_score, 4)
print 'Regresión de Ridge', round(ridge_score, 4)
print 'Random Forest', round(rf_score, 4)

Regresión lineal 0.2953
Regresión de Ridge 0.2953
Random Forest 0.4158


<br>
<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">La métrica de elección para evaluar los modelos es el R cuadrado (R2) que es mayor cuando el modelo puede explicar mejor la variabilidad en la respuesta. La regresión lineal y la regresión de Ridge logran explicar alrededor del 30% de la variación de los precios, mientras que el Random Forest explica el 41%. 
<br><br>
Estos modelos podrían mejorarse explorando otros modelos alternativos como las redes Neuronales. 
<br><br>
Los modelos Random Forest en sklearn proporciona la una funcionalidad que permite ver la importancia de la característica de nuestro modelo, que más explican la variación del precio. A continuación se muestra una tabla que contiene las 10 características o features principales que explican nuestro modelo:
</span>
</div>

In [9]:
imp_feature = [(x,y) for x,y in zip(X.columns, rf.feature_importances_)]
imp_feature = sorted(imp_feature, key=lambda x: x[1], reverse=True)[:9]

# Mostramos los valores
for feat, imp in imp_feature:
    print "El feature "+str(feat)+ " tiene un score de: "+str(round(imp, 4))


El feature room_type_Entire home/apt tiene un score de: 0.5046
El feature bathrooms tiene un score de: 0.1221
El feature latitude tiene un score de: 0.1086
El feature longitude tiene un score de: 0.0888
El feature accommodates tiene un score de: 0.051
El feature number_of_reviews tiene un score de: 0.0488
El feature review_scores_rating tiene un score de: 0.0238
El feature bedrooms tiene un score de: 0.0135
El feature instant_bookable tiene un score de: 0.0133




<br>
<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">Podemos ver que la caracteristica más importante arrojada por nuestro modelo de Random Forest es el tipo de habitación cuando el piso se alquila de su totalidad. Recordemos que esta variable era categórica y al pasasrse a una variable numérica se dividió en 3 variables distintas. 
<br><br>
Esto nos indica que existe una gran diferencia cuando un piso es alquilado con una habitación privada o compartida o cuando se alquila la propiedad completa. La cantidad de baños, la latitud y la longitud así como la cantidad de huéspedes parecen tener un efecto significativo en la variación del precio. 
<br><br>
Que en este punto aparezca como significante la latitud y la longitud nos dice que es importante la ubicación del piso, lo cuál tiene cierta lógica.
</span>
</div>

<br><div style="text-align: left;">
    <span style="font-weight: bold; color:#FF7674; font-family: 'Courier New'; font-size: 2em;">Impacto de las amenities o comodidades </span>
</div>



<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">Pueden las amenities mejorar mi modelo? Cuánto aportan al modelo final?
</span>
</div>

In [10]:
def has_pool(a):
    if 'Pool' in a:
        return 1
    else:
        return 0

listing['pool'] = listing['amenities'].apply(has_pool)

def Smoking(a):
    if 'Smoking' in a:
        return 1
    else:
        return 0

listing['Smoking'] = listing['amenities'].apply(has_pool)

def workspace(a):
    if 'workspace' in a:
        return 1
    else:
        return 0

listing['workspace'] = listing['amenities'].apply(has_pool)

def Internet(a):
    if 'Internet' in a:
        return 1
    else:
        return 0

listing['Internet'] = listing['amenities'].apply(has_pool)

In [11]:
features = ['Internet', 'workspace', 'Smoking', 'pool','neighbourhood_group_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 
            'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'instant_bookable', 'distancia_metro']
categorical = ['neighbourhood_group_cleansed', 'room_type']
numerical = ['Internet', 'workspace', 'Smoking', 'pool','accommodates', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'distancia_metro']


X = listing[features]
y = np.log(listing['price'])

In [12]:
print('Imputing Missing Values...')
Imp = Imputer(strategy='median')
X.loc[:,['Internet', 'workspace', 'Smoking', 'pool','accommodates', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'distancia_metro']] = Imp.fit_transform(X.loc[:, ['Internet', 'workspace', 'Smoking', 'pool','accommodates', 'bathrooms', 'bedrooms', 'latitude', 'longitude', 'number_of_reviews', 'review_scores_rating', 'distancia_metro']])


print('Scaling Numerical Variables...')
d = defaultdict(MinMaxScaler)
X.loc[:, numerical] = X.loc[:, numerical].apply(lambda x: d[x.name].fit_transform(x))

print('Creating Dummy Variables...')
X = pd.get_dummies(X, columns=categorical)

print('Splitting Data into Train/Test...')
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.2, random_state=1994)

print('Fit Linear Regression...')
lm = LinearRegression()
lm.fit(train_X, train_Y)

print('Fit Ridge Regression...')
ridge = Ridge()
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]}
gs = GridSearchCV(ridge, param_grid)
gs.fit(train_X, train_Y)
ridge = gs.best_estimator_
ridge.fit(train_X, train_Y)

print('Fit Random Forest...')
rf = RandomForestRegressor(n_estimators=1000, max_depth=16, min_samples_leaf=3)

rf.fit(train_X, train_Y)

Imputing Missing Values...
Scaling Numerical Variables...
Creating Dummy Variables...
Splitting Data into Train/Test...
Fit Linear Regression...
Fit Ridge Regression...
Fit Random Forest...


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=3,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [13]:
ridge_pred = ridge.predict(test_X)
ridge_score = r2_score(np.exp(test_Y), np.exp(ridge_pred))

lm_pred = lm.predict(test_X)
lm_score = r2_score(np.exp(test_Y), np.exp(lm_pred))

rf_pred = rf.predict(test_X)
rf_score = r2_score(np.exp(test_Y), np.exp(rf_pred))

In [14]:
print 'Linear Regression', round(lm_score, 4)
print 'Ridge Regression', round(ridge_score, 4)
print 'Random Forest', round(rf_score, 4)

Linear Regression 0.2992
Ridge Regression 0.2991
Random Forest 0.4148


In [15]:
# Match the feature importances with the column names and pick out the top 10
imp_feature = [(x,y) for x,y in zip(X.columns, rf.feature_importances_)]
imp_feature = sorted(imp_feature, key=lambda x: x[1], reverse=True)[:]
# Display the top 10 feature importances

for feat, imp in imp_feature:
    print "El feature "+str(feat)+ " tiene un score de: "+str(round(imp, 4))

El feature room_type_Entire home/apt tiene un score de: 0.5049
El feature bathrooms tiene un score de: 0.1223
El feature latitude tiene un score de: 0.1078
El feature longitude tiene un score de: 0.0875
El feature accommodates tiene un score de: 0.0509
El feature number_of_reviews tiene un score de: 0.0485
El feature review_scores_rating tiene un score de: 0.0238
El feature bedrooms tiene un score de: 0.0133
El feature instant_bookable tiene un score de: 0.0129
El feature neighbourhood_group_cleansed_Centro tiene un score de: 0.005
El feature neighbourhood_group_cleansed_Fuencarral - El Pardo tiene un score de: 0.0022
El feature room_type_Private room tiene un score de: 0.0021
El feature neighbourhood_group_cleansed_Salamanca tiene un score de: 0.002
El feature room_type_Shared room tiene un score de: 0.002
El feature neighbourhood_group_cleansed_Moncloa - Aravaca tiene un score de: 0.0019
El feature neighbourhood_group_cleansed_Chamberi tiene un score de: 0.0015
El feature neighbourho

<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">Como vemos, las amenities prácticamente no generan impacto alguno ni tampoco mejoran mi modelo...
</span>
</div>

<br>
<div style="text-align: left;">
    <span style="color:#2467C0; font-family: 'Courier New'; font-size: 1.5em;">La finalidad de este proyecto fue entender qué factores influyen en el precio de los pisos publicados en Airbnbs en Madrid. Una combinación de visualizaciones de datos y modelado nos dieron algunos resultados que nos permiten sacar algunas afirmaciones alrededor de esta pregunta.
<br><br>
En los datos existian varios anuncios que tenían precios anormalmente altos para el tipo de propiedad. Anuncios como esos pueden darse por publicaciones falsas o por una mala recolección de datos y pueden distorsionar los resultados mediante la introducción de falsas dependencias entre factores y precios altos. La labor de limpieza es FUNDAMENTAL.
<br><br>
Una de ellas es que parece haber una tendencia espacial en los precios, mostrada por la visualización de mapa y el modelo.
<br><br>
los pisos de rango medio a nivel de precios de concentran en la zona centro de Madrid y además una pequeña concentración en la zona del Barrio de Salamanca. Las propiedades más costosas se encuentran en en su mayoría en la zona norte de Madrid, mientras que en toda la ciudad se pueden conseguir pisos económicos.
<br><br>
El modelo generado tiene un muy amplio margen de mejora. Hay ciertos parámetros que no están definidos en nuestros datos pero que sin embargo -la lógica nos dice que- pueden afectar el precio de arrendamiento, como lo puede ser la decoración del apartamento, la limpieza y el cuidado general que tiene, 
<br><br>
Como trabajo futuro queda pendiente incluir mejoras en el modelo, nuevos features (inclusive incluir imágenes) y crear otros nuevos y más elaborados que permitan mejorar y optimizar nuestro resultado final.
</span>
</div>